In [5]:
import pandas as pd

df = pd.read_csv("100PlusInstance.csv", sep="\t", index_col="index")

In [6]:
from features_prompt import extract_features, extract_title

extend_frame = pd.DataFrame(columns=["Chỗ để xe hơi", "Đang cho thuê", "CSVC xung quanh", "Vấn đề pháp lý, sổ đỏ", "Số PN", "Số WC"])
error_index = []


for i in range(0, 5):
    print(i, end = "\t")
    features = extract_features(df["Description"][i])
    print(features)
    try:
        extend_frame.loc[i] = features
        print("Complete")
    except Exception as e:
        print(e)
        error_index.append(i)
    # print(df["Description"][i])

Quay lại xử lý các index bị lỗi
while(len(error_index) > 0):
    print(error_index)
    for i in error_index:
        print(i, end = "\t")
        ans, content = get_ans("Mô tả: '" + df["Description"][i] + "'", token = token)
        try:
            extend_frame.loc[i] = ans
            print("Complete")
            error_index.remove(i)
        except Exception as e:
            print(e)
    
print("Extract complete")


0	[True, True, True, True, 2, 1]
Complete
1	[False, False, True, False, 3, False]
Complete
2	[False, False, True, False, 2, 2]
Complete
3	[False, True, True, False, 0, 0]
Complete
4	[False, False, False, False, 0, 0]
Complete
Extract complete


In [9]:
# nối dataframe ban đầu với các features mới
df = pd.concat([df, extend_frame], axis=1) 

# combine các cột "Số phòng ngủ" và "Số phòng WC" với nhau
df["Số phòng ngủ"] = df["Số phòng ngủ"].apply(lambda x: 0 if x == "na" else x) # với giá trị là na thay bằng 0
df["Số phòng WC"] = df["Số phòng WC"].apply(lambda x: 0 if x == "na" else x)

df["Số phòng ngủ"] = df["Số phòng ngủ"].astype(float)
df["Số phòng WC"] = df["Số phòng WC"].astype(float)

def max_two_columns_PN(row):
    return max(row['Số phòng ngủ'], row['Số PN'])
def max_two_columns_WC(row):
    return max(row['Số phòng WC'], row['Số WC'])

df['Số phòng ngủ'] = df.apply(max_two_columns_PN, axis=1)
df['Số phòng WC'] = df.apply(max_two_columns_WC, axis=1)

df = df.drop(["Số PN", "Số WC"], axis = 1)

In [10]:

# df.to_csv("100PlusInstance_updated.csv", sep="\t", index=False)

df.head(3)

,Title,Price,Links,Diện tích (m2),Số phòng ngủ,Số phòng WC,Location,Description,Chỗ để xe hơi,Đang cho thuê,CSVC xung quanh,"Vấn đề pháp lý, sổ đỏ"
0,Chưa tới 30tr/m2 - Hàng ngộp bank BAO ĐẦU TƯ ...,Giá bán\nDưới 500 triệu\nTừ 500 - 800 triệu\nT...,https://batdongsan.vn/chua-toi-30trm2-hang-ngo...,150,2.0,1.0,"('10.7763897', '106.7011391')",Mô tả:\n+ Thông số cực đẹp 5 x 30m (150m2)\...,True,True,True,True
1,"Bán nhà HXH Âu Cơ Phường 9 Tân Bình, 51m2 3 Tầ...",3899000000 tỷ,https://batdongsan.vn/ban-nha-hxh-au-co-phuong...,51,3.0,0.0,NaN,"Bán nhà HXH Âu Cơ Phường 9 Tân Bình, 51m2 3 Tầ...",False,False,True,False
2,"SÁT MẶT TIỀN PHAN ĐĂNG LƯU, PHƯỜNG 7, PHÚ NHUẬ...",5.5 tỷ,https://batdongsan.vn/sat-mat-tien-phan-dang-l...,45,2.0,2.0,NaN,"- Kết cấu: 1 trệt 1 lầu BTCT; 2 PN, 2 WC. Phòn...",False,False,True,False


In [30]:
def check_instance(index):
    print(df["Description"][index])
    print()
    print("Tiêu đề: " + df["Title"][index])
    # print("\n")
    # print(df.iloc[index])

check_instance(0)

Mô tả:
+ Thông số cực đẹp 5 x 30m (150m2)
+ Nhà cấp 4, 2PN, toilet, phòng khách, bếp, sân sau trồng cây.
+ Nhà gần sát Mặt tiền, hẻm nhựa xe hơi tránh.
+ Khu vực tập trung nhiều Khu dân cư, các Dự án lớn, thu hút đông đảo các Nhà Đầu Tư lớn, Huyện Nhà Bè hướng đến mục tiêu trở thành Thành phố trực thuộc TP.Hồ Chí Minh.
+ Giá chưa tới 30tr/m2 bao đầu tư.
+ Sổ đang vay bank 2.2 tỷ, pháp lý chuẩn, chủ cần bán gấp.
☎️ Thiện chí alo em Hiền gặp chính chủ thương lượng, miễn trung gian.

Tiêu đề: Chưa tới 30tr/m2 - Hàng ngộp bank BAO ĐẦU TƯ - HẠ CHÀO 1.1 TỶ
